In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import os
import numpy as np
import pandas as pd
import re
from os.path import basename, splitext
import time
import pickle
import json
from datetime import datetime

# from solver import Instance

from argparse import Namespace
import sys
sys.path.append('../solver')
# from solver_output import practice_print
from solver_rostering_new_baseline import run_baseline_solver_output

/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
city = 'paris'

# fixed params
RM = 1.5
GM = 0.8
h_min = 32
h_max = 48
max_n_diff = 3

refresh = False

In [3]:
list_city_dt = []

for file_in in os.listdir(r'../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#Subset to frankfurt
list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find(city) > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']


In [4]:
def load_shift(shift_file, OC, RM, GM, model, max_n_shifts=None):
    with open(shift_file, 'r') as file:
        data = json.load(file)
        df_shifts = pd.DataFrame(data)
    df_shifts = df_shifts[(df_shifts['outsourcing_cost_multiplier']==OC)&(df_shifts['regional_multiplier']==RM)&(df_shifts['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_shifts = df_shifts[df_shifts['model']==model]
    #partflex
    else:
        df_shifts = df_shifts[(df_shifts['model']==model)&(df_shifts['max_n_shifts']==max_n_shifts)]
    df_shifts.reset_index(drop = True, inplace=True)
    df_shifts = df_shifts[['region','shifts_start','shifts_end']]
    dict_shifts = df_shifts.set_index('region').to_dict(orient='index')
    return dict_shifts

In [5]:
def load_workforce(workforce_file, OC, RM, GM, model, max_n_shifts=None):
    with open(workforce_file, 'r') as file:
        data = json.load(file)
        df_workforce = pd.DataFrame(data)
    df_workforce = df_workforce[(df_workforce['outsourcing_cost_multiplier']==OC)&(df_workforce['regional_multiplier']==RM)&(df_workforce['global_multiplier']==GM)]
    #fixed or flex
    if model in ['fixed','flex']:
        df_workforce = df_workforce[df_workforce['model']==model]
    #partflex
    else:
        df_workforce = df_workforce[(df_workforce['model']==model)&(df_workforce['max_n_shifts']==max_n_shifts)]
    df_workforce.reset_index(drop = True, inplace=True)
    return (int(df_workforce['workforce_size_region0'].tolist()[0])-1)


In [6]:
# for base_file in list_city_dt:
for base_file in ['lyon_db=0.50']:

    #create inputs to run a trial workforce size
    weekday_ = f'{base_file}_dt=doublepeak.json'
    weekend_ = f'{base_file}_dt=uniform.json'

    instance_file_weekday = f"../instances/{weekday_}"
    instance_file_weekend = f"../instances/{weekend_}"

    shift_file_weekday = f"../shifts/{weekday_}"
    shift_file_weekend = f"../shifts/{weekend_}"

    expand_workforce_to_regions=True
    workforce_dict = {}

    new_shift_weekend = None
    new_shift_weekday = None
    new_workforce = None
    dict_results = {}

    for OC in [1.2, 1.5, 1.8, 2.0, 2.5]:
    # for OC in [1.2]:

        new_shift_weekend = None
        new_shift_weekday = None
        new_workforce = None
        dict_results = {}

        # for model in ['flex',  'partflex', 'fixed']:
        for (model, max_n_shifts) in [('fixed',None),('partflex',2),('flex',None),('partflex',3),('partflex',4)]:

            if max_n_shifts == None:
                results_file = f"../rerun_baseline/{base_file}_OC={OC}_model={model}.pkl"
            else:
                results_file = f"../rerun_baseline/{base_file}_OC={OC}_model={model}_max_n_shifts={max_n_shifts}.pkl"

            old_results = dict_results

            # Check if shift pattern is already solved
            old_workforce = new_workforce
            new_workforce = load_workforce(f'../workforce_size/{base_file}.json', OC, RM, GM, model, max_n_shifts)
            old_shift_weekend = new_shift_weekend
            old_shift_weekday = new_shift_weekday
            new_shift_weekend = load_shift(shift_file_weekend, OC, RM, GM, model, max_n_shifts)
            new_shift_weekday = load_shift(shift_file_weekday, OC, RM, GM, model, max_n_shifts)

            if old_shift_weekend == new_shift_weekend and old_shift_weekday == new_shift_weekday and old_workforce == new_workforce:
                old_results['model']  = [model]
                old_results['max_n_shifts'] = [max_n_shifts]

                print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, already solved. Saving previous results')

                # Save previous results
                with open(results_file, 'wb') as file:
                    pickle.dump(old_results, file)
                continue
            
            start_time = time.time()
            dict_results = \
                run_baseline_solver_output(model, instance_file_weekday, 
                                                    shift_file_weekday, instance_file_weekend, 
                                                    shift_file_weekend, workforce_dict, 
                                                    OC, RM, GM, h_min, h_max, max_n_diff, 
                                                    max_n_shifts, expand_workforce_to_regions=True)
            
            end_time = time.time()
            run_time = (end_time - start_time)
            
            # Save results
            with open(results_file, 'wb') as file:
                pickle.dump(dict_results, file)
            print(f'Base: {base_file}, OC: {OC}, model: {model}-{max_n_shifts}, run_time: {run_time}')



Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-06
Base: lyon_db=0.50, OC: 1.2, model: fixed-None, run_time: 2.495487689971924
Base: lyon_db=0.50, OC: 1.2, model: partflex-2, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.2, model: flex-None, run_time: 5.456137180328369
Base: lyon_db=0.50, OC: 1.2, model: partflex-3, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.2, model: partflex-4, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.5, model: fixed-None, run_time: 2.3850557804107666
Base: lyon_db=0.50, OC: 1.5, model: partflex-2, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.5, model: flex-None, run_time: 2.370166540145874
Base: lyon_db=0.50, OC: 1.5, model: partflex-3, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.5, model: partflex-4, already solved. Saving previous results
Base: lyon_db=0.50, OC: 1.8, model: fixed-None, run_time: 2.4632651805877686
B